In [1]:
# Import libraries
import psycopg2
from transformers import MarianMTModel, MarianTokenizer

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Traslation model configuration
modelo = "Helsinki-NLP/opus-mt-es-en"
tokenizer = MarianTokenizer.from_pretrained(modelo)
model = MarianMTModel.from_pretrained(modelo)

In [3]:
# BBDD connection
conn = psycopg2.connect(
    dbname="manga_db",
    user="user_master",
    password="PassMaster97",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

In [4]:
# Select mangas with Spanish synopsis
cur.execute("SELECT id, synopsis_es FROM mangas_base WHERE synopsis_es IS NOT NULL AND (synopsis IS NULL OR synopsis = '')")
mangas = cur.fetchall()

In [5]:
# Loop through each manga and translate
for manga_id, texto_es in mangas:
    try:
        # Tokenization and translation
        tokens = tokenizer(texto_es, return_tensors="pt", padding=True, truncation=True)
        traduccion = model.generate(**tokens)
        texto_en = tokenizer.decode(traduccion[0], skip_special_tokens=True)

        # Update the database with the translated synopsis
        cur.execute(
            "UPDATE mangas_base SET synopsis = %s WHERE id = %s",
            (texto_en, manga_id)
        )
        print(f"[OK] Manga ID {manga_id} translated correctly.")

    except Exception as e:
        print(f"[ERROR] Manga ID {manga_id} failed: {e}")

[OK] Manga ID 114046 translated correctly.
[OK] Manga ID 114070 translated correctly.
[OK] Manga ID 114095 translated correctly.
[OK] Manga ID 114146 translated correctly.
[OK] Manga ID 114178 translated correctly.
[OK] Manga ID 114193 translated correctly.
[OK] Manga ID 114259 translated correctly.
[OK] Manga ID 114304 translated correctly.
[OK] Manga ID 114305 translated correctly.
[OK] Manga ID 114306 translated correctly.
[OK] Manga ID 114307 translated correctly.
[OK] Manga ID 114301 translated correctly.
[OK] Manga ID 114302 translated correctly.
[OK] Manga ID 114303 translated correctly.
[OK] Manga ID 114084 translated correctly.
[OK] Manga ID 114097 translated correctly.
[OK] Manga ID 114108 translated correctly.
[OK] Manga ID 114125 translated correctly.
[OK] Manga ID 114045 translated correctly.
[OK] Manga ID 114047 translated correctly.
[OK] Manga ID 114048 translated correctly.
[OK] Manga ID 114049 translated correctly.
[OK] Manga ID 114296 translated correctly.
[OK] Manga 

In [6]:
# Confirm changes and close connection
conn.commit()
cur.close()
conn.close()